In [2]:
!pip install scikeras --upgrade
!pip install scikit-learn --upgrade
!pip install keras --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 112.7 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sklearn-compat 0.1.3 requires scikit-learn<1.7,>=1.2, but you have scikit-learn 1.7.0 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.7 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
^C


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import matplotlib.pyplot as plt
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import L2


veri=pd.read_csv("/content/diabetes_dataset.csv")

le=LabelEncoder()
veri["Outcome"]=le.fit_transform(veri["Outcome"])

y=veri["Outcome"]
x=veri.drop(columns="Outcome")

sc=StandardScaler()
x=sc.fit_transform(x)

x_train, x_test, y_train, y_test= train_test_split(x,y,test_size=0.2, random_state=0)
x_train, x_val, y_train, y_val= train_test_split(x_train, y_train, test_size=0.1, random_state=0)

def modelkur(units=64, learning_rate=0.01, hidden_layers=2, reg=L2(0.01), dropout_rate=0.2):

  model=Sequential()
  model.add(Dense(units=units, input_dim=x_train.shape[1], activation="relu", kernel_regularizer=reg))

  for i in range(hidden_layers):
    model.add(Dense(units, activation="relu", kernel_regularizer=reg))
    model.add(Dropout(dropout_rate))

  model.add(Dense(1, activation="sigmoid"))
  model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate), metrics=["accuracy"])

  return model

parametreler={
    "model__units":[16,32,64],
    "model__learning_rate":[0.001,0.01],
    "model__hidden_layers":[2,3],
    "model__reg":[None, L2(0.001), L2(0.01)],
    "model__dropout_rate":[0.0,0.2],
    "epochs":[20,50]
}

model=KerasClassifier(model=modelkur, verbose=0)

grid=GridSearchCV(estimator=model, param_grid=parametreler, cv=5)
gridsonuc=grid.fit(x_train, y_train)
bestparam=gridsonuc.best_params_

bestmodel=modelkur(units=bestparam["model__units"],
                   learning_rate=bestparam["model__learning_rate"],
                   reg=bestparam["model__reg"],
                   dropout_rate=bestparam["model__dropout_rate"],
                   hidden_layers=bestparam["model__hidden_layers"])


output=model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=bestparam["epochs"], verbose=0)







fig,ax=plt.subplots(1,2,figsize=(25,10))

ax[0].plot(output.history["loss"], label="Training Loss")
ax[0].plot(output.history["val_loss"], label="Validation Loss")
ax[0].set_title("Graph of Loss")
ax[0].set_ylabel("Loss")
ax[0].set_xlabel("Epochs")
ax[0].legend()


ax[1].plot(output.history["accuracy"], label="Accuracy Loss")
ax[1].plot(output.history["val_accuracy"], label="Accuracy Loss")
ax[1].set_title("Graph of Accuracy")
ax[1].set_ylabel("Accuracy")
ax[1].set_xlabel("Epochs")
ax[1].legend()

plt.show()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/lo